## General imports

The `inverted_index_colab` import requires the `inverted_index_colab.py` file. You should upload the file and then run this cell.

In [1]:
import sys
import math
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Installing, importing, and initializing PySpark


The following code installs PySpark and its dependencies in Colab. In addition, we install GraphFrames, which is a PySpark package for dealing with graphs in a distributed fashion. Colab notebooks run on a single machine so we will work in local mode, i.e. there is no cluster of machines and both the master and worker processes run on a single machine. This will help us debug and iron out the code we will use in the second half of this assignment on an actual cluster mode in GCP. 

The installation in the next cell should take about 1 minute in a fresh environment. Don't worry about going  over the 90 seconds limit for the assignment because in our testing enviroment all of these requirements will already be met. 

In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
--2022-01-07 22:26:32--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 13.32.87.119, 13.32.87.63, 13.32.87.68, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|13.32.87.119|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

As mentioned above, all wiki dumps were preprocessed and placed in a shared google storage bucket. To access the files in Colab, you will first have to authenticate with google storage, and then copy the data to your local environment. In GCP, the storage will already be mounted on the cluster machines and we will show you how to access it. 

In [5]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [6]:
import pickle
from inverted_index_colab import *
import os

os.environ["GCLOUD_PROJECT"] = "assignment3-334422"


from google.cloud import storage
index_name = "index.pkl"
bucket_name = "newshanishpack"
client = storage.Client()
blobs = client.list_blobs(bucket_name)

def readIndex(bucket_name):
  f_name = index_name
  gs_name = f"postings_gcp/{f_name}"
  storage_client = storage.Client()
  bucket = storage_client.bucket(bucket_name)
  blob = bucket.blob(gs_name)
  blob.download_to_filename(f'./{f_name}')
  f = open(f_name, 'rb')
  return pickle.load(f)

inverted = readIndex(bucket_name)

In [7]:
#!gsutil -m cp -r \
 # "gs://newshanishpack/gcp/" \
 # .

In [13]:
!gsutil -m cp -r \
  gs://newshanishpack/postings_gcp "content/" \
  .

Copying gs://newshanishpack/postings_gcp/0_000.bin...
Copying gs://newshanishpack/postings_gcp/0_005.bin...
Copying gs://newshanishpack/postings_gcp/0_001.bin...
Copying gs://newshanishpack/postings_gcp/0_002.bin...
Copying gs://newshanishpack/postings_gcp/0_003.bin...
Copying gs://newshanishpack/postings_gcp/0_006.bin...
Copying gs://newshanishpack/postings_gcp/0_007.bin...
Copying gs://newshanishpack/postings_gcp/0_004.bin...
Copying gs://newshanishpack/postings_gcp/0_008.bin...
Copying gs://newshanishpack/postings_gcp/0_009.bin...
Copying gs://newshanishpack/postings_gcp/0_010.bin...
Copying gs://newshanishpack/postings_gcp/0_011.bin...
Copying gs://newshanishpack/postings_gcp/0_012.bin...
Copying gs://newshanishpack/postings_gcp/0_013.bin...
Copying gs://newshanishpack/postings_gcp/0_014.bin...
Copying gs://newshanishpack/postings_gcp/0_015.bin...
Copying gs://newshanishpack/postings_gcp/0_016.bin...
Copying gs://newshanishpack/postings_gcp/0_017.bin...
Copying gs://newshanishpack/

In [9]:
#!gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

In [10]:
#inverted.DL
j=0
for i in inverted.posting_lists_iter():
  #if j==14:
   # break
  print(i)
  j+=1
#words, pls = zip(*inverted.posting_lists_iter())


('gibbus', [(17998, 1), (45930, 1), (56579, 1), (189164, 1), (300898, 1), (322239, 1), (453600, 1), (453745, 1), (508173, 1), (855774, 2), (1586337, 1), (1783345, 1), (2073576, 1), (2707517, 1), (4015120, 1), (4341076, 1), (5260228, 1), (5407683, 1), (5694531, 1), (5880172, 1), (5893812, 1), (5993846, 1), (6025118, 2), (8997609, 3), (10578011, 1), (10947294, 1), (11208536, 1), (11379576, 2), (11789635, 1), (11986475, 1), (12508033, 2), (12612639, 1), (12654955, 1), (12654957, 4), (12702370, 1), (12710436, 1), (12710581, 1), (12831667, 1), (13331017, 1), (13564290, 1), (13865676, 2), (14040197, 1), (14672359, 1), (14937988, 1), (16778276, 1), (17622696, 4), (18673638, 2), (18922827, 1), (19206200, 1), (19988821, 1), (20141176, 7), (20788287, 2), (21094499, 1), (21905745, 1), (22221389, 1), (22235646, 1), (22937733, 1), (23769801, 1), (25738572, 1), (25946605, 2), (26029634, 1), (26037924, 1), (26038093, 1), (27093063, 1), (27093577, 1), (27373911, 1), (28128551, 1), (28167517, 2), (2844

FileNotFoundError: ignored

In [ ]:
def generate_query_tfidf_vector(query_to_search,index):
    """ 
    Generate a vector representing the query. Each entry within this vector represents a tfidf score.
    The terms representing the query will be the unique terms in the index.

    We will use tfidf on the query as well. 
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the query.    

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.
    
    Returns:
    -----------
    vectorized query with tfidf scores
    """
    epsilon = .0000001
    total_vocab_size = len(index.df.keys())
    Q = np.zeros((total_vocab_size))
    term_vector = list(index.df.keys())    
    counter = Counter(query_to_search)
    for token in np.unique(query_to_search):
        if token in index.df.keys(): #avoid terms that do not appear in the index.
            tf = counter[token]/len(query_to_search) # term frequency divded by the length of the query
            df = index.df[token]
            idf = math.log(len(index.DL)/(df+epsilon),10) #smoothing
            try:
                ind = term_vector.index(token)
                Q[ind] = tf*idf
            except:
                pass
    return Q

In [ ]:
def get_candidate_documents_and_scores(query_to_search,index,words,pls):
    """
    Generate a dictionary representing a pool of candidate documents for a given query. This function will go through every token in query_to_search
    and fetch the corresponding information (e.g., term frequency, document frequency, etc.') needed to calculate TF-IDF from the posting list.
    Then it will populate the dictionary 'candidates.'
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the document.
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: generator for working with posting.
    Returns:
    -----------
    dictionary of candidates. In the following format:
                                                               key: pair (doc_id,term)
                                                               value: tfidf score. 
    """
    candidates = {}
    N = len(index.DL)        
    for term in np.unique(query_to_search):        
        if term in words:            
            list_of_doc = pls[words.index(term)]                        
            normlized_tfidf = [(doc_id,(freq/index.DL[str(doc_id)])*math.log(N/index.df[term],10)) for doc_id, freq in list_of_doc]           
                        
            for doc_id, tfidf in normlized_tfidf:
                candidates[(doc_id,term)] = candidates.get((doc_id,term), 0) + tfidf               
        
    return candidates

In [ ]:
def generate_document_tfidf_matrix(query_to_search,index,words,pls):
    """
    Generate a DataFrame `D` of tfidf scores for a given query. 
    Rows will be the documents candidates for a given query
    Columns will be the unique terms in the index.
    The value for a given document and term will be its tfidf score.
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: generator for working with posting.
    Returns:
    -----------
    DataFrame of tfidf scores.
    """
    
    total_vocab_size = len(index.df)
    candidates_scores = get_candidate_documents_and_scores(query_to_search,index,words,pls) #We do not need to utilize all document. Only the docuemnts which have corrspoinding terms with the query.
    unique_candidates = np.unique([doc_id for doc_id, freq in candidates_scores.keys()])
    D = np.zeros((len(unique_candidates), total_vocab_size))
    D = pd.DataFrame(D)
    
    D.index = unique_candidates
    D.columns = index.df.keys()

    for key in candidates_scores:
        tfidf = candidates_scores[key]
        doc_id, term = key    
        D.loc[doc_id][term] = tfidf

    return D

In [ ]:
def cosine_similarity(D,Q):
    """
    Calculate the cosine similarity for each candidate document in D and a given query (e.g., Q).
    Generate a dictionary of cosine similarity scores 
    key: doc_id
    value: cosine similarity score
    
    Parameters:
    -----------
    D: DataFrame of tfidf scores.

    Q: vectorized query with tfidf scores
    
    Returns:
    -----------
    dictionary of cosine similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: cosine similarty score.
    """
    # YOUR CODE HERE
    mydict = {}
    i=0
    IDlist = [x for x in D.index]
    D1 = D.to_numpy()
    for row in D1:
      first=(np.dot(row,Q))
      normQ=np.linalg.norm(Q)
      normRow = np.linalg.norm(row)
      res=first/(normRow*normQ)
      mydict[IDlist[i]]=res
      i+=1

    return mydict
    

In [ ]:
def get_top_n(sim_dict,N=3):
    """ 
    Sort and return the highest N documents according to the cosine similarity score.
    Generate a dictionary of cosine similarity scores 
   
    Parameters:
    -----------
    sim_dict: a dictionary of similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: similarity score. We keep up to 5 digits after the decimal point. (e.g., round(score,5))

    N: Integer (how many documents to retrieve). By default N = 3
    
    Returns:
    -----------
    a ranked list of pairs (doc_id, score) in the length of N.
    """
    res = sorted([(doc_id,score) for doc_id, score in sim_dict.items()], key = lambda x: x[1],reverse=True)
    if len(res)>N:
      res = res[:N]

    return res

In [ ]:
def get_pagerank(wiki_ids, index):
  res = []
  for id in wiki_ids:
    res.append(pr_results.vertices.filter(pr_results.vertices.id == id).select("pagerank").collect()[0][0])
  return res

def search_body(query, index):
  res = []
  Q = generate_query_tfidf_vector(query, index)
  D = generate_document_tfidf_matrix(query,index,words,pls)
  cosine_similarity_dict = cosine_similarity(D,Q)
  top = get_top_n(cosine_similarity_dict,100)
  for doc in top:
    title = index.id_title[doc[0]]
    res.append((doc[0],title))
  return res

def search_title(query, index): 
  # dict that key is docid and val is the sum of appearances of all the words in the query in the specific doc
  res = []
  mydict = {} 
  for q in query:
    for k in index.posting_lists_iter():
        for docid, appearances in k[1]:
          if docid not in mydict.keys():
            mydict[docid] = appearances
          else:
            mydict[docid] = mydict[docid]+appearances
        break
  appearances_per_doc = [(key,val) for key,val in mydict.items()]
  sorted_appearances = sorted(appearances_per_doc, key = lambda x: x[1],reverse=True)
  for doc in sorted_appearances:
    title = index.id_title[doc[0]]
    res.append((doc[0],title))
  return res


print(search_body(["why","do","men","have","nipples"], inverted))